In [1]:
import pandas as pd
import json
import re
import src.cleaning as cl

In [2]:
df = pd.read_csv("./data/shots_v1.csv")

In [3]:
df.head()

,Unnamed: 0,0,1
0,0,1,"[{'A. Della Valle': ['#local-out', '104.418306..."
1,1,2,"[{'J. Zyskowski': ['#local-in', '24.75', '126...."
2,2,3,"[{'D. DiLeo': ['#local-out', '166.684342478713..."
3,3,4,"[{'J. Rousselle': ['#local-in', '24.75', '134...."
4,4,5,"[{'J. Vaulet': ['#local-dunk', '24.75', '141.2..."


In [4]:
df.rename(columns = {"0": "Jornada"}, inplace = True)
df.drop(columns = ["Unnamed: 0"], inplace = True)

In [5]:
names = cl.shots_per_player(df)

In [6]:
for elem in names:
    if elem == " ":
        names.remove(elem)

In [7]:
len(names)

37285

In [8]:
jornadas, prueba = cl.cumulat_shots(df)


In [9]:
len(jornadas)

33

In [10]:
# As we had problems trying to append the first gameday number of shots, we append it before looping. 
shots_per_jornada = [1157]
for i in range(len(jornadas)-1):
    shots_per_jornada.append(jornadas[i+1]-jornadas[i])

In [11]:
shots_per_jornada

[1157,
 1133,
 1141,
 1133,
 1133,
 1125,
 1135,
 1109,
 1112,
 1101,
 1146,
 1110,
 1107,
 1163,
 1173,
 1138,
 1051,
 1116,
 1140,
 1145,
 1158,
 985,
 1149,
 1157,
 1132,
 1159,
 1128,
 1141,
 1138,
 1147,
 1136,
 1166,
 1121]

In [12]:
num_jornada = list(range(1, 35))
num_jornada = [str(x) for x in num_jornada]

In [13]:
final_jornada = sum([[s] * n for s, n in zip(num_jornada, shots_per_jornada)], [])

In [14]:
#Finally we get a list with values from 1 to 34 (gamedays played) according to when every shot was made. 
len(final_jornada)

37285

### Now we are gonna divide the list "prueba" in order to get data for each three elements. 

### Here we divide the information contained in "prueba" in three lists. One to know if the shot was made or not, as a local or as a visitor. Another for its coordinate x and the last one for its coordinate y.

In [16]:
made = []
for i in range(0, len(prueba), 3):
    made.append(prueba[i])
    
coord_x = []
for i in range(1, len(prueba), 3):
    coord_x.append(prueba[i])
    
coord_y = []
for i in range(2, len(prueba), 3):
    coord_y.append(prueba[i])
    
cositas = list(zip(names, final_jornada, made, coord_x, coord_y))
df = pd.DataFrame(cositas)
df.rename(columns = {0: "name", 1: "jornada", 2: "shot", 3: "coord_x", 4: "coord_y"}, inplace = True)

In [17]:
df.head()

,name,jornada,shot,coord_x,coord_y
0,J. Zyskowski,1,#local-in,24.75,126.41666666666667
1,J. Zyskowski,1,#local-in,24.75,23.883333333333333
2,J. Zyskowski,1,#local-out,25.705345316934725,23.883333333333333
3,J. Zyskowski,1,#local-in,27.616035950804164,137.86666666666667
4,J. Zyskowski,1,#local-in,27.12899716177862,129.81666666666666


In [18]:
df["coord_x"] = pd.to_numeric(df["coord_x"], downcast="float")
df["coord_y"] = pd.to_numeric(df["coord_y"], downcast="float")
df["jornada"] = pd.to_numeric(df["jornada"], downcast="integer")

In [19]:
df_visitor = df.loc[(df["shot"] == "#visitor-in") | (df["shot"] == "#visitor-out") | (df["shot"] == "#visitor-dunk")]
df_local = df.loc[(df["shot"] == "#local-in") | (df["shot"] == "#local-out") | (df["shot"] == "#local-dunk")]
df_visitor["coord_x"] = abs(df_visitor["coord_x"] - 513)
df = pd.concat([df_local, df_visitor])
df["name"] = df["name"].str.strip()

<ipython-input-19-6cad6d911b17>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_visitor["coord_x"] = abs(df_visitor["coord_x"] - 513)


In [20]:
df.to_csv("./data/database_shots.csv")

## Here we import our csv with every shot.

In [21]:
every_shot = pd.read_csv("./data/df_tiros_polygon.csv")

In [22]:
every_shot.sort_values('PLAYER_NAME', ascending = True, inplace = True)
every_shot.drop(columns = ["Unnamed: 0", "index_right", "geometry", "coord_x", "coord_y"], inplace = True)

In [23]:
type_shots = ["in", "out"]

for i in type_shots:
    every_shot.shot = every_shot.shot.apply(lambda x: i if i in x else x)
        
every_shot.shot = every_shot.shot.apply(lambda x: 'in' if 'dunk' in x else x)

In [24]:
every_shot["PLAYER_NAME"] = every_shot["PLAYER_NAME"].str.replace('á', 'a').str.replace('à', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('Á', 'A').str.replace('Í', 'I').str.replace('ñ', 'n')
every_shot["PLAYER_NAME"] = every_shot["PLAYER_NAME"].str.replace('AJ Slaughter', 'A. Slaughter').str.replace('V. Gerun', 'V. Herun').str.replace('C. Pacheco', 'C. D. S. Pacheco').str.replace('F. Dos Anjos', 'F. D. Anjos').str.replace('P. Dime', 'M. Dime').str.replace('J. Gonzalez', 'C. Gonzalez').str.replace('W. Tavares', 'E. Tavares').str.replace('X. X. Lopez-Arostegui', 'X. Lopez-Arostegui').str.replace('A. Balcerowski', 'O. Balcerowski').str.replace('I. Llovet', 'N. Llovet')

<ipython-input-24-5b9f9fa30a91>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  every_shot["PLAYER_NAME"] = every_shot["PLAYER_NAME"].str.replace('AJ Slaughter', 'A. Slaughter').str.replace('V. Gerun', 'V. Herun').str.replace('C. Pacheco', 'C. D. S. Pacheco').str.replace('F. Dos Anjos', 'F. D. Anjos').str.replace('P. Dime', 'M. Dime').str.replace('J. Gonzalez', 'C. Gonzalez').str.replace('W. Tavares', 'E. Tavares').str.replace('X. X. Lopez-Arostegui', 'X. Lopez-Arostegui').str.replace('A. Balcerowski', 'O. Balcerowski').str.replace('I. Llovet', 'N. Llovet')


### The variable made represents all the shots which were in during the game.

In [25]:
cositas = every_shot.groupby(["name_left", "PLAYER_NAME","shot"]).count()
cositas.reset_index(inplace = True)

In [26]:
made = cositas.loc[cositas["shot"] == "in"]
made.drop(columns = ["shot"], inplace = True)
made.rename(columns = {"jornada": "shots_made"}, inplace = True)

/home/pablofdez/miniconda3/envs/ironhack/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/home/pablofdez/miniconda3/envs/ironhack/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [69]:
made

,name_left,PLAYER_NAME,shots_made
0,polygon1,A. Abalde,1
3,polygon1,A. Albicy,2
5,polygon1,A. Avramovic,2
8,polygon1,A. Brown,5
10,polygon1,A. Cvetkovic,1
...,...,...,...
4819,polygon9,Y. Ndoye,2
4823,polygon9,Y. Sima,2
4824,polygon9,Z. Dragic,1
4826,polygon9,Z. Maronka,1


In [28]:
#Aquí sacamos los totales
total = every_shot.groupby(["name_left", "PLAYER_NAME"]).count()
total.reset_index(inplace = True)
total.drop(columns = ["shot"], inplace = True)
total.rename(columns = {"jornada": "shots_attempted"}, inplace = True)

In [70]:
total

,name_left,PLAYER_NAME,shots_attempted
0,polygon1,A. Abalde,2
1,polygon1,A. Abrines,1
2,polygon1,A. Albicy,3
3,polygon1,A. Avramovic,2
4,polygon1,A. Barrera,1
...,...,...,...
2773,polygon9,Y. Ouattara,2
2774,polygon9,Y. Sima,2
2775,polygon9,Z. Dragic,3
2776,polygon9,Z. Maronka,1


### Time to merge both DataFrames

In [71]:
zonas = pd.merge(total, made, how = "left", left_on = ["name_left", "PLAYER_NAME"], right_on = ["name_left", "PLAYER_NAME"])

In [73]:
zonas.fillna(0, inplace = True)

In [89]:
zonas

,name_left,PLAYER_NAME,shots_attempted,shots_made
0,polygon1,A. Abalde,2,1.0
1,polygon1,A. Abrines,1,0.0
2,polygon1,A. Albicy,3,2.0
3,polygon1,A. Avramovic,2,2.0
4,polygon1,A. Barrera,1,0.0
...,...,...,...,...
2773,polygon9,Y. Ouattara,2,0.0
2774,polygon9,Y. Sima,2,2.0
2775,polygon9,Z. Dragic,3,1.0
2776,polygon9,Z. Maronka,1,1.0


### Importamos DataFrame de estadísticas generales

In [75]:
average_stats = pd.read_csv("./data/acb_stats.csv")

In [76]:
average_stats.drop(columns = ["Unnamed: 0"], inplace = True)
average_stats.rename(columns = {"Player": "PLAYER_NAME"}, inplace = True)
average_stats.sort_values('PLAYER_NAME', ascending = True, inplace = True)
average_stats.head()

,#,PLAYER_NAME,Team,GP,MIN,FGM,FGA,FG%,3PM,3PA,...,DRB,REB,AST,STL,BLK,PTS,Pos,Height,Weight,Age
124,1,A.J. Slaughter,GCN,22,24.3,5.0,10.2,0.484,1.7,4.2,...,1.9,2.3,3.7,1.1,0.2,13.7,G,6-4,185,33
174,5,Aaron Doornekamp,CAN,31,25.6,2.6,6.1,0.431,2.0,4.6,...,2.9,3.5,1.3,0.8,0.1,8.4,PF,6-7,212,35
274,15,Aaron Jones,BBB,8,19.7,1.6,3.9,0.419,0.4,1.0,...,1.5,3.8,1.0,0.5,0.4,4.1,F,6-9,220,27
281,2,Achille Polonara,CAJ,33,26.5,4.2,8.8,0.479,1.4,4.1,...,3.8,5.5,1.9,1.1,0.5,11.3,PF,6-8,198,29
23,11,Adam Hanga,FCB,31,16.1,2.2,3.7,0.578,1.1,2.0,...,2.8,3.4,2.1,0.8,0.4,6.2,SF,6-7,200,31


In [77]:
average_stats["PLAYER_NAME"] = average_stats["PLAYER_NAME"].str.replace('Osas Ehigitor', 'Osas Ehigiator').str.replace('T. Mcfadden', 'T. McFadden').str.replace('D. Dileo', 'D. DiLeo').str.replace('Tunde Olumuyiwa', 'B. Olumuyiwa').str.replace('K. C. Jr.', 'K. Cheatham').str.replace('T. Bray', 'TJ Bray').str.replace('J. Barea', 'JJ Barea').str.replace('S. V. Rossom', 'S. Van Rossom').str.replace('J. W. Iii', 'J. Webb').str.replace('M. Kingsley', 'K. Moses').str.replace('A. D. Valle', 'A. Della Valle').str.replace('D. Rendl', 'J. Randle').str.replace('R. S. Miguel', 'R. San Miguel').str.replace('T. Campbell', 'TJ Campbell').str.replace('F. S. Emeterio', 'F. San Emeterio')

<ipython-input-77-5453df86f208>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  average_stats["PLAYER_NAME"] = average_stats["PLAYER_NAME"].str.replace('Osas Ehigitor', 'Osas Ehigiator').str.replace('T. Mcfadden', 'T. McFadden').str.replace('D. Dileo', 'D. DiLeo').str.replace('Tunde Olumuyiwa', 'B. Olumuyiwa').str.replace('K. C. Jr.', 'K. Cheatham').str.replace('T. Bray', 'TJ Bray').str.replace('J. Barea', 'JJ Barea').str.replace('S. V. Rossom', 'S. Van Rossom').str.replace('J. W. Iii', 'J. Webb').str.replace('M. Kingsley', 'K. Moses').str.replace('A. D. Valle', 'A. Della Valle').str.replace('D. Rendl', 'J. Randle').str.replace('R. S. Miguel', 'R. San Miguel').str.replace('T. Campbell', 'TJ Campbell').str.replace('F. S. Emeterio', 'F. San Emeterio')


In [78]:
cambio_nombre = cl.change_names(average_stats)

In [79]:
average_stats["mergeo"] = cambio_nombre
average_stats["mergeo"] = average_stats["mergeo"].str.replace('O. Ehigitor', 'O. Ehigiator').str.replace('T. Mcfadden', 'T. McFadden').str.replace('D. Dileo', 'D. DiLeo').str.replace('T. Olumuyiwa', 'B. Olumuyiwa').str.replace('K. C. Jr.', 'K. Cheatham').str.replace('T. Bray', 'TJ Bray').str.replace('J. Barea', 'JJ Barea').str.replace('S. V. Rossom', 'S. Van Rossom').str.replace('J. W. Iii', 'J. Webb').str.replace('M. Kingsley', 'K. Moses').str.replace('A. D. Valle', 'A. Della Valle').str.replace('D. Rendl', 'J. Randle').str.replace('R. S. Miguel', 'R. San Miguel').str.replace('T. Campbell', 'TJ Campbell').str.replace('F. S. Emeterio', 'F. San Emeterio')
average_stats.head()

<ipython-input-79-2ae1efd74a31>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  average_stats["mergeo"] = average_stats["mergeo"].str.replace('O. Ehigitor', 'O. Ehigiator').str.replace('T. Mcfadden', 'T. McFadden').str.replace('D. Dileo', 'D. DiLeo').str.replace('T. Olumuyiwa', 'B. Olumuyiwa').str.replace('K. C. Jr.', 'K. Cheatham').str.replace('T. Bray', 'TJ Bray').str.replace('J. Barea', 'JJ Barea').str.replace('S. V. Rossom', 'S. Van Rossom').str.replace('J. W. Iii', 'J. Webb').str.replace('M. Kingsley', 'K. Moses').str.replace('A. D. Valle', 'A. Della Valle').str.replace('D. Rendl', 'J. Randle').str.replace('R. S. Miguel', 'R. San Miguel').str.replace('T. Campbell', 'TJ Campbell').str.replace('F. S. Emeterio', 'F. San Emeterio')


,#,PLAYER_NAME,Team,GP,MIN,FGM,FGA,FG%,3PM,3PA,...,REB,AST,STL,BLK,PTS,Pos,Height,Weight,Age,mergeo
124,1,A.J. Slaughter,GCN,22,24.3,5.0,10.2,0.484,1.7,4.2,...,2.3,3.7,1.1,0.2,13.7,G,6-4,185,33,A. Slaughter
174,5,Aaron Doornekamp,CAN,31,25.6,2.6,6.1,0.431,2.0,4.6,...,3.5,1.3,0.8,0.1,8.4,PF,6-7,212,35,A. Doornekamp
274,15,Aaron Jones,BBB,8,19.7,1.6,3.9,0.419,0.4,1.0,...,3.8,1.0,0.5,0.4,4.1,F,6-9,220,27,A. Jones
281,2,Achille Polonara,CAJ,33,26.5,4.2,8.8,0.479,1.4,4.1,...,5.5,1.9,1.1,0.5,11.3,PF,6-8,198,29,A. Polonara
23,11,Adam Hanga,FCB,31,16.1,2.2,3.7,0.578,1.1,2.0,...,3.4,2.1,0.8,0.4,6.2,SF,6-7,200,31,A. Hanga


In [82]:
final = average_stats.merge(zonas, how = 'left', left_on = 'mergeo', right_on = 'PLAYER_NAME')

In [83]:
final

,#,PLAYER_NAME_x,Team,GP,MIN,FGM,FGA,FG%,3PM,3PA,...,PTS,Pos,Height,Weight,Age,mergeo,name_left,PLAYER_NAME_y,shots_attempted,shots_made
0,1,A.J. Slaughter,GCN,22,24.3,5.0,10.2,0.484,1.7,4.2,...,13.7,G,6-4,185,33,A. Slaughter,polygon10,A. Slaughter,8.0,5.0
1,1,A.J. Slaughter,GCN,22,24.3,5.0,10.2,0.484,1.7,4.2,...,13.7,G,6-4,185,33,A. Slaughter,polygon11,A. Slaughter,5.0,4.0
2,1,A.J. Slaughter,GCN,22,24.3,5.0,10.2,0.484,1.7,4.2,...,13.7,G,6-4,185,33,A. Slaughter,polygon12,A. Slaughter,3.0,1.0
3,1,A.J. Slaughter,GCN,22,24.3,5.0,10.2,0.484,1.7,4.2,...,13.7,G,6-4,185,33,A. Slaughter,polygon2,A. Slaughter,3.0,3.0
4,1,A.J. Slaughter,GCN,22,24.3,5.0,10.2,0.484,1.7,4.2,...,13.7,G,6-4,185,33,A. Slaughter,polygon3,A. Slaughter,7.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3119,6,Zoran Dragic,CAJ,31,18.0,3.0,6.0,0.500,0.7,2.2,...,8.6,SG,6-5,200,31,Z. Dragic,polygon6,Z. Dragic,46.0,31.0
3120,6,Zoran Dragic,CAJ,31,18.0,3.0,6.0,0.500,0.7,2.2,...,8.6,SG,6-5,200,31,Z. Dragic,polygon7,Z. Dragic,54.0,35.0
3121,6,Zoran Dragic,CAJ,31,18.0,3.0,6.0,0.500,0.7,2.2,...,8.6,SG,6-5,200,31,Z. Dragic,polygon8,Z. Dragic,21.0,5.0
3122,6,Zoran Dragic,CAJ,31,18.0,3.0,6.0,0.500,0.7,2.2,...,8.6,SG,6-5,200,31,Z. Dragic,polygon9,Z. Dragic,3.0,1.0


In [283]:
final = average_stats.merge(zona_1, how = 'left', left_on='mergeo', right_on='mergeo')
final = final.merge(zona_2, how = 'left', left_on='mergeo', right_on='mergeo')
final = final.merge(zona_3, how = 'left', left_on='mergeo', right_on='mergeo')
final = final.merge(zona_4, how = 'left', left_on='mergeo', right_on='mergeo')
final = final.merge(zona_5, how = 'left', left_on='mergeo', right_on='mergeo')
final = final.merge(zona_6, how = 'left', left_on='mergeo', right_on='mergeo')
final = final.merge(zona_7, how = 'left', left_on='mergeo', right_on='mergeo')
final = final.merge(zona_8, how = 'left', left_on='mergeo', right_on='mergeo')
final = final.merge(zona_9, how = 'left', left_on='mergeo', right_on='mergeo')
final = final.merge(zona_10, how = 'left', left_on='mergeo', right_on='mergeo')
final = final.merge(zona_11, how = 'left', left_on='mergeo', right_on='mergeo')
final = final.merge(zona_12, how = 'left', left_on='mergeo', right_on='mergeo')

NameError: name 'average_stats' is not defined

In [1265]:
final.drop(columns = ["mergeo"], inplace = True)
final.fillna(0, inplace= True)
final = final.loc[final["GP"] > 7]
final.head()

In [1275]:
final.to_csv("./data/final_average_stats.csv")

### Try with per40

In [256]:
per = pd.read_csv("./data/per40_stats.csv")
per.drop(columns = ["Unnamed: 0"], inplace = True)
per.rename(columns = {"Player": "PLAYER_NAME"}, inplace = True)
per.sort_values('PLAYER_NAME', ascending = True, inplace = True)
per.head()

,#,PLAYER_NAME,Team,GP,MIN,FGM,FGA,FG%,3PM,3PA,...,DRB,REB,AST,STL,BLK,PTS,Pos,Height,Weight,Age
124,3,A.J. Slaughter,GCN,22.0,24.3,8.2,16.9,0.484,2.8,6.9,...,3.1,3.8,6.1,1.8,0.3,22.6,G,6-4,185,33
178,11,Aaron Doornekamp,CAN,31.0,25.6,4.1,9.5,0.431,3.2,7.2,...,4.5,5.4,2.0,1.3,0.2,13.1,PF,6-7,212,35
275,19,Aaron Jones,BBB,8.0,19.7,3.3,7.9,0.419,0.8,2.0,...,3.0,7.6,2.0,1.0,0.8,8.4,F,6-9,220,27
283,6,Achille Polonara,CAJ,33.0,26.5,6.4,13.3,0.479,2.1,6.2,...,5.8,8.2,2.9,1.7,0.8,17.1,PF,6-8,198,29
22,12,Adam Hanga,FCB,31.0,16.1,5.4,9.3,0.578,2.6,4.9,...,7.1,8.5,5.2,2.1,1.0,15.5,SF,6-7,200,31


In [257]:
per["PLAYER_NAME"] = per["PLAYER_NAME"].str.replace('Osas Ehigitor', 'Osas Ehigiator').str.replace('T. Mcfadden', 'T. McFadden').str.replace('D. Dileo', 'D. DiLeo').str.replace('Tunde Olumuyiwa', 'B. Olumuyiwa').str.replace('K. C. Jr.', 'K. Cheatham').str.replace('T. Bray', 'TJ Bray').str.replace('J. Barea', 'JJ Barea').str.replace('S. V. Rossom', 'S. Van Rossom').str.replace('J. W. Iii', 'J. Webb').str.replace('M. Kingsley', 'K. Moses').str.replace('A. D. Valle', 'A. Della Valle').str.replace('D. Rendl', 'J. Randle').str.replace('R. S. Miguel', 'R. San Miguel').str.replace('T. Campbell', 'TJ Campbell').str.replace('F. S. Emeterio', 'F. San Emeterio')

<ipython-input-257-41f2df5b8f06>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  per["PLAYER_NAME"] = per["PLAYER_NAME"].str.replace('Osas Ehigitor', 'Osas Ehigiator').str.replace('T. Mcfadden', 'T. McFadden').str.replace('D. Dileo', 'D. DiLeo').str.replace('Tunde Olumuyiwa', 'B. Olumuyiwa').str.replace('K. C. Jr.', 'K. Cheatham').str.replace('T. Bray', 'TJ Bray').str.replace('J. Barea', 'JJ Barea').str.replace('S. V. Rossom', 'S. Van Rossom').str.replace('J. W. Iii', 'J. Webb').str.replace('M. Kingsley', 'K. Moses').str.replace('A. D. Valle', 'A. Della Valle').str.replace('D. Rendl', 'J. Randle').str.replace('R. S. Miguel', 'R. San Miguel').str.replace('T. Campbell', 'TJ Campbell').str.replace('F. S. Emeterio', 'F. San Emeterio')


In [258]:
cambio_nombre = cl.change_names(per)

In [259]:
per["mergeo"] = cambio_nombre
per["mergeo"] = per["mergeo"].str.replace('O. Ehigitor', 'O. Ehigiator').str.replace('T. Mcfadden', 'T. McFadden').str.replace('D. Dileo', 'D. DiLeo').str.replace('T. Olumuyiwa', 'B. Olumuyiwa').str.replace('K. C. Jr.', 'K. Cheatham').str.replace('T. Bray', 'TJ Bray').str.replace('J. Barea', 'JJ Barea').str.replace('S. V. Rossom', 'S. Van Rossom').str.replace('J. W. Iii', 'J. Webb').str.replace('M. Kingsley', 'K. Moses').str.replace('A. D. Valle', 'A. Della Valle').str.replace('D. Rendl', 'J. Randle').str.replace('R. S. Miguel', 'R. San Miguel').str.replace('T. Campbell', 'TJ Campbell').str.replace('F. S. Emeterio', 'F. San Emeterio')
per.head()

<ipython-input-259-671be5decc93>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  per["mergeo"] = per["mergeo"].str.replace('O. Ehigitor', 'O. Ehigiator').str.replace('T. Mcfadden', 'T. McFadden').str.replace('D. Dileo', 'D. DiLeo').str.replace('T. Olumuyiwa', 'B. Olumuyiwa').str.replace('K. C. Jr.', 'K. Cheatham').str.replace('T. Bray', 'TJ Bray').str.replace('J. Barea', 'JJ Barea').str.replace('S. V. Rossom', 'S. Van Rossom').str.replace('J. W. Iii', 'J. Webb').str.replace('M. Kingsley', 'K. Moses').str.replace('A. D. Valle', 'A. Della Valle').str.replace('D. Rendl', 'J. Randle').str.replace('R. S. Miguel', 'R. San Miguel').str.replace('T. Campbell', 'TJ Campbell').str.replace('F. S. Emeterio', 'F. San Emeterio')


,#,PLAYER_NAME,Team,GP,MIN,FGM,FGA,FG%,3PM,3PA,...,REB,AST,STL,BLK,PTS,Pos,Height,Weight,Age,mergeo
124,3,A.J. Slaughter,GCN,22.0,24.3,8.2,16.9,0.484,2.8,6.9,...,3.8,6.1,1.8,0.3,22.6,G,6-4,185,33,A. Slaughter
178,11,Aaron Doornekamp,CAN,31.0,25.6,4.1,9.5,0.431,3.2,7.2,...,5.4,2.0,1.3,0.2,13.1,PF,6-7,212,35,A. Doornekamp
275,19,Aaron Jones,BBB,8.0,19.7,3.3,7.9,0.419,0.8,2.0,...,7.6,2.0,1.0,0.8,8.4,F,6-9,220,27,A. Jones
283,6,Achille Polonara,CAJ,33.0,26.5,6.4,13.3,0.479,2.1,6.2,...,8.2,2.9,1.7,0.8,17.1,PF,6-8,198,29,A. Polonara
22,12,Adam Hanga,FCB,31.0,16.1,5.4,9.3,0.578,2.6,4.9,...,8.5,5.2,2.1,1.0,15.5,SF,6-7,200,31,A. Hanga


In [284]:
final = per.merge(zona_1, how = 'left', left_on='mergeo', right_on='mergeo')
final = final.merge(zona_2, how = 'left', left_on='mergeo', right_on='mergeo')
final = final.merge(zona_3, how = 'left', left_on='mergeo', right_on='mergeo')
final = final.merge(zona_4, how = 'left', left_on='mergeo', right_on='mergeo')
final = final.merge(zona_5, how = 'left', left_on='mergeo', right_on='mergeo')
final = final.merge(zona_6, how = 'left', left_on='mergeo', right_on='mergeo')
final = final.merge(zona_7, how = 'left', left_on='mergeo', right_on='mergeo')
final = final.merge(zona_8, how = 'left', left_on='mergeo', right_on='mergeo')
final = final.merge(zona_9, how = 'left', left_on='mergeo', right_on='mergeo')
final = final.merge(zona_10, how = 'left', left_on='mergeo', right_on='mergeo')
final = final.merge(zona_11, how = 'left', left_on='mergeo', right_on='mergeo')
per_final = final.merge(zona_12, how = 'left', left_on='mergeo', right_on='mergeo')

In [285]:
per_final.drop(columns = ["mergeo"], inplace = True)
per_final.fillna(0, inplace= True)
per_final = per_final.loc[per_final["GP"] > 7]
per_final.head()

,#,PLAYER_NAME,Team,GP,MIN,FGM,FGA,FG%,3PM,3PA,...,zona8_attempted,zona8_made,zona9_attempted,zona9_made,zona10_attempted,zona10_made,zona11_attempted,zona11_made,zona12_attempted,zona12_made
0,3,A.J. Slaughter,GCN,22.0,24.3,8.2,16.9,0.484,2.8,6.9,...,32.0,13.0,16.0,7.0,8.0,5.0,5.0,4.0,3.0,1.0
1,11,Aaron Doornekamp,CAN,31.0,25.6,4.1,9.5,0.431,3.2,7.2,...,31.0,13.0,4.0,1.0,25.0,16.0,8.0,2.0,6.0,4.0
2,19,Aaron Jones,BBB,8.0,19.7,3.3,7.9,0.419,0.8,2.0,...,4.0,1.0,2.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
3,6,Achille Polonara,CAJ,33.0,26.5,6.4,13.3,0.479,2.1,6.2,...,39.0,12.0,7.0,3.0,14.0,3.0,5.0,3.0,2.0,1.0
4,12,Adam Hanga,FCB,31.0,16.1,5.4,9.3,0.578,2.6,4.9,...,19.0,9.0,2.0,2.0,6.0,3.0,2.0,1.0,5.0,1.0


In [286]:
per_final.to_csv("./data/per40_final_stats.csv")